In [ ]:
import os
from typing import List
import time

import matplotlib.pyplot as plt
import numpy as np
import pyproj
import rasterio
import rasterio.features
import shapely
import torch

from torchgeo.datasets.utils import download_url
from torchgeo.transforms import indices
from tqdm import tqdm

### Formato de carpetas generada por QGIS al descargar imágenes
La carpeta `L2A_T19HDC_A038131_20221010T144648_2022-10-10_con` contiene 10 bandas en archivos separados.
<pre>└── <font color="#12488B"><b>L2A_T19HDC_A038131_20221010T144648_2022-10-10_con</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B02.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B03.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B04.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B05.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B06.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B07.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B08.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B11.tif</b></font>
    ├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B12.tif</b></font>
    └── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_B8A.tif</b></font>
</pre>

### Después de procesar la imágenes el resultado sería:
<pre><font color="#12488B"><b>/ruta/para/guardar/imagenes</b></font>
├── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_all.tif</b></font>
└── <font color="#A347BA"><b>RT_T19HDC_A038131_20221010T144648_rgb.tif</b></font>
</pre>
La imagen que termina en `_all.tif` contiene todas las bandas, y la que termina en `_rgb` solo las bandas `B04`, `B03` y `B02`.

In [2]:
def extract_window(path: str, url: str, geometry: shapely.geometry.Polygon) -> None:
    """Recorta y guarda los archivos .tiff dada las coords."""

    with rasterio.open(url) as ds:
        transform = pyproj.Transformer.from_crs("epsg:4326", ds.crs)
        # polygon to bbox (xyxy)
        bbox = rasterio.features.bounds(geometry)
        # convert bbox to source CRS (xyxy)
        coords = [
            transform.transform(bbox[3], bbox[0]),
            transform.transform(bbox[1], bbox[2]),
        ]
        # convert coords to pixel coords (xyxy)
        pcoords = [
            ds.index(coords[0][0], coords[0][1]),
            ds.index(coords[1][0], coords[1][1]),
        ]
        # convert bbox (xyxy) -> (xxyy)
        bbox_xxyy = ((pcoords[0][0], pcoords[1][0]), (pcoords[0][1], pcoords[1][1]))
        window = rasterio.windows.Window.from_slices(*bbox_xxyy)

        # Copy and update tiff metadata for windowed image
        metadata = ds.meta.copy()
        metadata.update(
            dict(
                height=window.height,
                width=window.width,
                transform=rasterio.windows.transform(window, ds.transform),
                compress="DEFLATE",
            )
        )

        # Write to geotiff
        with rasterio.open(path, "w", **metadata) as ds_windowed:
            ds_windowed.write(ds.read(1, window=window), 1)
            

def stack(root: str, alias: str, bands: List[str]) -> None:
    """Junta las bandas separadas en una imagen EPSG:4326 CRS."""
    files = [os.path.join(root + band) for band in bands]
    with rasterio.open(files[0]) as ds:
        metadata = ds.meta
        metadata["count"] = len(files)

    with rasterio.open(f"{root}{alias}.tif", "w", **metadata) as dst:
        for i, f in enumerate(files, start=1):
            with rasterio.open(f) as src:
                dst.write_band(i, src.read(1))


In [3]:
#geometry = shapely.geometry.Polygon(
#    [
#        [-69.607866, -28.135092],
#        [-69.440833, -28.228162],
#        [-69.64045, -28.51795],
#        [-69.814809, -28.417246],
#    ]
#)

# coord del poligono
geometry = shapely.geometry.Polygon(
    [
        [-69.82037354, -28.11996841],
        [-69.42983246, -28.11996841],
        [-69.82037354, -28.52018356],
        [-69.42983246, -28.52018356],
    ]
)



# ruta donde se encuetran todas las carpetas con la imagenes .tif
# suelen terminar con "_con"
path_root = '/media/chr/Linux/glaciares/'

# se define el path donde se guardaran las imagenes finales
# en este caso en la ruta: /media/chr/Linux/glaciares/dataset/polygon_images_bands_parallel/
path_final = path_root + 'dataset/polygon_images_bands/'

# se crea un arreglo que contiene todas las carpetas
folders = [path for path in os.listdir(path_root) if path.endswith("_con")]

# Todas las bandas
bands_all = ["B04.tif", "B03.tif", "B02.tif", "B05.tif", "B06.tif", "B07.tif", "B08.tif", "B8A.tif", "B11.tif", "B12.tif"]

# Solo se utilizan las bandas RGB para crear las imagenes .jpg
bands_rgb = ["B04.tif", "B03.tif", "B02.tif"] # RGB        

In [ ]:
for folder in tqdm(folders[:5]):
    image_name = os.listdir(path_root + folder)[0][:-7]
    try:
        for band in bands_all:
            # Se extrae solo la parte del poligono
            extract_window(path_final + image_name + band, path_root + folder + '/' + image_name + band, geometry)
        # Se crea .tif con las bandas RGB
        stack(path_final + image_name, "rgb", bands_rgb)
        # Se crea .tif con todas las bandas
        stack(path_final + image_name, "all", bands_all)
    except:
        # Quizas falta alguna banda .tif
        print('Error en: ', path_root + folder + '/' + image_name + band)

 60%|███████████████████████████                  | 3/5 [03:50<02:41, 80.58s/it]

Se está trabajando en una opción paralelizable